In [1241]:
%pylab inline
import re
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [1242]:
myfiles = !ls \data
print myfiles

['20160116S1-FORZA_16_Asics_Black.txt', '20160116S1-LAVA_North_16_Adidas.txt', '20160116S1-SCVC_16_1.txt', '20160116S2-FORZA_16_Asics_Black.txt', '20160116S2-LAVA_North_16_Adidas.txt', '20160116S2-SCVC_16_1.txt', '20160116S3-SCVC_16_1.txt']


In [1243]:
skillList = ['Sv','Batt', 'A', 'S','D','B','BA', 'P', 'H']

data = !cat data/20160116S2-LAVA_North_16_Adidas.txt | grep '^[0-9].*'
lineup = !cat data/20160116S2-LAVA_North_16_Adidas.txt | grep '^lineup.*' | cut -d' ' -f2
lineup = lineup[0].split('|')

print lineup

rotationStats = []
playerSkillResults = []
assistList = []
j = 0
for i in range(0,len(data)):
    #get data for this play
    play_data = data[i].split('|')
    begin_score = play_data[0]
    
    #get resulting score from play
    if i+1 < len(data):
        result_score = data[i+1].split('|')[0]
        del play_data[0]
    else:
        result_score = data[i].split('|')[0]
        
    #determine whether the lineup has changed and change it if you need to
    subs = [s for s in play_data if 'sub' in s]
    for c in subs:
        lineup_change = c.replace('sub ','').split('/')
        lineup = [lineup_change[0] if x == lineup_change[1] else x for x in lineup]
        
    #Find Assists
    index = [i for i, s in enumerate(play_data) if 'Ak' in s]
    if len(index) == 1: 
        if re.findall(r'S', play_data[index[0]-1]):
            play_data[index[0]-1] = play_data[index[0]-1].replace('S','Sa')
        else:
            assister = re.findall(r'\d+', play_data[index[0]-1])
            play_data.insert(index[0], assister[0]+'Sa' )
    
    #determine whether or not we are in a new rotation
    server = str([re.findall(r'\d+',s) for s in play_data if 'SV' in s]).strip("[['']]")
    
    try:
        if server != last_server and server != '': j += 1
    except NameError:
        j=0
    
    last_server = server
    if j%6 != 0:
        rotation = j%6
    else:
        rotation = 6

    #get active player list for play (i.e. players who touched the ball on the play)
    active = []
    for a in [s for s in play_data if 'sub' not in s]:
        all_digits = re.findall(r'\d+', a)
        if len(all_digits) > 0 and i+1 < len(data):
            active.append(all_digits[0])
            player = all_digits[0]
            skill = str(re.findall(r'[A-Z]+', a)).strip("['']")
            if len(all_digits) > 1:
                result = all_digits[1]
            else:
                result = str(re.findall(r'[\+\-aek]+', a)).strip("['']")
        playerSkillResults.append([player, skill , result, 1])
    #print active
    
    if i+1 < len(data):
        home_begin_score = int(begin_score.split('-')[0])
        opponent_begin_score = int(begin_score.split('-')[1])
        home_result_score = int(result_score.split('-')[0])
        opponent_result_score = int(result_score.split('-')[1])
        point = home_result_score - home_begin_score - opponent_result_score + opponent_begin_score 
        
        rotationStats.append([begin_score, rotation, play_data, result_score, point, lineup, list(set(active))])
        
df = pd.DataFrame.from_records(rotationStats)
df.columns = ['start_score', 'rotation', 'play_action','result_score','point', 'lineup', 'active', ]
df

['7', '15', '11', '2', '3', '4']


,start_score,rotation,play_action,result_score,point,lineup,active
0,0-0,6,"[4P3, 2Sa, 7Ak]",1-0,1,"[7, 15, 11, 2, 3, 4]","[2, 4, 7]"
1,1-0,1,"[sub 14/7, 14SVe]",1-1,-1,"[14, 15, 11, 2, 3, 4]",[14]
2,1-1,1,"[3P2, 14Sa, 23Ak]",2-1,1,"[14, 15, 11, 2, 3, 4]","[3, 14, 23]"
3,2-1,2,"[sub 25/3, 15SVe]",2-2,-1,"[14, 15, 11, 2, 25, 4]",[15]
4,2-2,2,[23P0],2-3,-1,"[14, 15, 11, 2, 25, 4]",[23]
5,2-3,2,"[4P1, 11S, 25A, 23D, 14S, 11A, 15D, 14S, 23A, ...",3-3,1,"[14, 15, 11, 2, 25, 4]","[11, 25, 15, 14, 23, 4]"
6,3-3,3,[11SVa],4-3,1,"[14, 15, 11, 2, 25, 4]",[11]
7,4-3,3,[11SVe],4-4,-1,"[14, 15, 11, 2, 25, 4]",[11]
8,4-4,3,"[25P3, 14Sa-, 25Ak]",5-4,1,"[14, 15, 11, 2, 25, 4]","[25, 14]"
9,5-4,4,"[sub 23/22, sub 7/14, 2SV]",6-4,1,"[7, 15, 11, 2, 25, 4]",[2]


In [1244]:
result_df = pd.DataFrame.from_records(playerSkillResults, columns=('player', 'skill', 'result', 'total'))
result_df = result_df.groupby(['player','skill', 'result']).count()
pd.options.display.multi_sparse = False
result_df = result_df.reset_index()

## Attacks
#### Definition: strategic hit by a player with the intent of scoring a point
#### Key point: intent – if the player is not trying to score a point, THEN THERE IS NO ATTACK (and, as a result, no dig)
#### Three outcomes of an attack:
* Kill: an attack that directly leads to a point (Ak)
* Attack error: an attack that directly results in a point for the opposing team (Ae)
* Ball stays in play (a.k.a. “zero attack”) (A)

#### Two subjective evaluations of an attack:
* Good - puts the other team out of rotation (+)
* Bad - equivalent of freeball (-)

#### Examples of Attack error:
* Ball hit out of bounds
* Ball hit into the net
* Ball is blocked and results in a point
* Four-hit violation
* Center-line fault
* Back-row attack
* Illegal contact
* Antenna fault
* 10-foot-line violation    

#### Hitting Percentage:  (kills – errors)/total attacks: (K-E / TA)

It is NOT an attack if:
* There is a bad set 
* A player passes ball over the net to keep the ball in play
* Freeballs
* If a dig flies over the net and drops for a point, it is a dig AND a kill
* Reception attempt to kill - Ditto
* Back-row blocks
* Kills from overpasses - These look like blocks – they’re NOT. They’re kills.

In [1245]:
#ATTACK
attackDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'A']
total = attackDF.groupby('player').sum()
total.columns = ['Total Attacks']
kills = attackDF[attackDF['result'] == 'k'].groupby('player').sum()
kills.columns = ['Kills']
error =  attackDF[attackDF['result'] == 'e'].groupby('player').sum()
error.columns = ['Errors']
good = attackDF[attackDF['result'] == '+'].groupby('player').sum()
good.columns = ['Good Attack']
bad = attackDF[attackDF['result'] == '+'].groupby('player').sum()
bad.columns = ['Bad Attack']
neutral = attackDF[attackDF['result'] == ''].groupby('player').sum()
neutral.columns = ['Fair Attack']

attack_stats = pd.concat([kills, error, good, bad, neutral, total], axis='1', join='outer')
attack_stats = attack_stats.fillna(0)
attack_stats['Hitting Percentage'] = (attack_stats.Kills - attack_stats.Errors) / attack_stats['Total Attacks']

print 'ATTACK', 
attack_stats.sort_values('Hitting Percentage', ascending=False)

ATTACK

,Kills,Errors,Good Attack,Bad Attack,Fair Attack,Total Attacks,Hitting Percentage
11,2,0,0,0,1,3,0.666667
9,2,0,0,0,1,3,0.666667
23,2,0,0,0,3,5,0.400000
15,1,0,0,0,2,3,0.333333
25,3,1,0,0,3,7,0.285714
5,1,1,0,0,0,2,0.000000
7,1,1,0,0,3,5,0.000000
14,0,1,0,0,0,1,-1.000000


## Setting/Assists
#### Definition (assist): when a player passes, sets or digs ball to teammate who gets a kill	
#### Key point: it does not HAVE to be a set or be on the second contact
#### Three outcomes of a set:
* Assist (Sa)
* Assist error/ball handing error (Se)
    -when an assist attempt results in an error – 99 percent of the time, this will be a ball handling error and it will be called on a setter on second contact
* Zero assist (does not lead to a kill) (S)

#### Two subjective evaluations of an set:
* Good - Location, height and speed are accurate (+)
* Bad - Location, height and speed are inaccurate (-)

Examples of ball handling errors:
* Double hit
* Thrown ball
* Lifted ball

Tips
Don’t forget your back row! 
* ANY contact that leads to a kill is an assist – it doesn’t have to be a traditional “set”	
Sets that go over the net
* If it falls for a point, it’s a KILL (and an attack)
* If it goes out of bounds, it’s an attack error


In [1246]:
#SETTING
settingDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'S']

total = settingDF.groupby('player').sum()
total.columns = ['Total Sets']
assist = settingDF[(settingDF['result'].isin(['a', 'a+']))].groupby('player').sum()
assist.columns = ['Assist']
good = settingDF[(settingDF['result'].isin(['+', 'a+']))].groupby('player').sum()
good.columns = ['Good']
bad = settingDF[(settingDF['result'].isin(['-', 'a-']))].groupby('player').sum()
bad.columns = ['Bad']
error = settingDF[settingDF['result'] == 'e'].groupby('player').sum()
error.columns = ['Error']
neutral = settingDF[settingDF['result'] == ''].groupby('player').sum()
neutral.columns = ['Neutral']

setting_stats = pd.concat([assist, error, good, bad, neutral, total], axis='1', join='outer')
setting_stats = setting_stats.fillna(0)
setting_stats

,Assist,Error,Good,Bad,Neutral,Total Sets
11,0,0,0,0,1,1
14,3,0,3,2,3,10
2,6,0,6,0,6,13
23,1,0,0,0,0,1
4,1,0,0,0,1,2
9,0,0,0,0,1,1


In [1247]:
#SERVING
serveDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'SV']
total = serveDF.groupby('player').sum()
aces = serveDF[serveDF['result'] == 'a'].groupby('player').sum()
aces.columns = ['aces']
error =  serveDF[serveDF['result'] == 'e'].groupby('player').sum()
error.columns = ['error']
good = serveDF[serveDF['result'] == '+'].groupby('player').sum()
good.columns = ['good']
neutral = serveDF[serveDF['result'] == ''].groupby('player').sum()
neutral.columns = ['neutral']

serve_stats = pd.concat([aces, error, good, neutral, total], axis='1', join='outer')
serve_stats = serve_stats.fillna(0)
serve_stats['Serve Pct'] = (serve_stats.aces - serve_stats.error) / serve_stats.total

print 'SERVE'
serve_stats.sort_values('Serve Pct', ascending=False)

SERVE


,aces,error,good,neutral,total,Serve Pct
3,2,0,3,3,8,0.25
11,1,1,0,0,2,0.00
15,1,1,0,3,5,0.00
2,0,0,0,2,2,0.00
4,0,0,0,1,1,0.00
5,0,0,0,1,1,0.00
14,1,2,0,1,4,-0.25


In [1248]:
#PASSING
passDF = result_df[['player', 'result', 'total']][result_df['skill'] == 'P']
total = passDF.groupby('player').sum()
error = passDF[passDF['result'] == '0'].groupby('player').sum()
error.columns = ['0']
ones =  passDF[passDF['result'] == '1'].groupby('player').sum()
ones.columns = ['1']
twos = passDF[passDF['result'] == '2'].groupby('player').sum()
twos.columns = ['2']
threes = passDF[passDF['result'] == '3'].groupby('player').sum()
threes.columns = ['3']

passing_stats = pd.concat([error, ones, twos, threes, total], axis='1', join='outer')
passing_stats = passing_stats.fillna(0)
passing_stats['Passing Weighted'] = ((passing_stats['0']*0 )+ (passing_stats['1']*1) + (passing_stats['2']*2 )
                                + (passing_stats['3']*3)) / (passing_stats['total'])

print 'PASS', 
passing_stats.sort_values('Passing Weighted', ascending=False)

PASS

,0,1,2,3,total,Passing Weighted
15,0,0,0,3,3,3.000000
7,0,0,0,1,1,3.000000
3,0,0,1,2,3,2.666667
4,2,3,0,8,13,2.076923
25,0,1,2,1,4,2.000000
11,0,1,0,0,1,1.000000
23,1,0,0,0,1,0.000000


In [1249]:
positivePointDict = {}
totalPointsPlayed = {}
totalActivePointsPlayed = {}
positiveActivePointsPlayed = {}

for idx, col in df[['point','lineup']].iterrows():
    for a in col[1]:
        if a in positivePointDict.keys():
            positivePointDict[a] += col[0]
            totalPointsPlayed[a] += 1
        else:
            positivePointDict[a] = col[0]
            totalPointsPlayed[a] = 1
            
for idx, col in df[['point','active']].iterrows():
    for a in col[1]:
        if a in positiveActivePointsPlayed.keys():
            positiveActivePointsPlayed[a] += col[0]
            totalActivePointsPlayed[a] += 1
        else:
            positiveActivePointsPlayed[a] = col[0]
            totalActivePointsPlayed[a] = 1

positivePointDF = pd.DataFrame.from_dict(positivePointDict, orient='index')
positivePointDF.columns = ['Net Positive Points']
totalPointsPlayedDF = pd.DataFrame.from_dict(totalPointsPlayed, orient='index')
totalPointsPlayedDF.columns = ['Total Points']
totalActivePointsPlayedDF = pd.DataFrame.from_dict(totalActivePointsPlayed, orient='index')
totalActivePointsPlayedDF.columns = ['Total Active Points']
positiveActivePointsPlayedDF = pd.DataFrame.from_dict(positiveActivePointsPlayed, orient='index')
positiveActivePointsPlayedDF.columns = ['Net Positive Active Points']

ppp = pd.concat([positivePointDF,totalPointsPlayedDF,totalActivePointsPlayedDF,positiveActivePointsPlayedDF], axis='1', join='outer')
ppp = ppp.fillna(0)
ppp['Positive Point Pct'] = ppp['Net Positive Points']/ppp['Total Points']
ppp['Active Positive Point Pct'] = ppp['Net Positive Active Points']/ppp['Total Active Points']
ppp = ppp.fillna(0)
ppp.sort_values('Active Positive Point Pct', ascending=False)

,Net Positive Points,Total Points,Total Active Points,Net Positive Active Points,Positive Point Pct,Active Positive Point Pct
3,4,35,10,6,0.114286,0.600000
9,0,0,6,2,0.000000,0.333333
11,3,46,4,1,0.065217,0.250000
7,3,21,8,2,0.142857,0.250000
15,3,46,13,3,0.065217,0.230769
23,0,0,5,1,0.000000,0.200000
14,0,25,14,2,0.000000,0.142857
24,0,0,1,0,0.000000,0.000000
25,-1,11,8,0,-0.090909,0.000000
4,3,46,15,0,0.065217,0.000000


In [1250]:
roster = {25: 'Sophia', 14: 'Sabrina', 9: 'Olivia', 11: 'Morgan', 5: 'Lindsey', 15: 'Katie', 7: 'Mikayla', 4: 'Ellie', 2: 'Jordan', 6:'Emma', 3:'Lexi', 23:'Mekenzi'}
